# Validação do Pipeline
Este notebook executa e valida cada etapa do pipeline de processamento de dados e geração de resultados. Cada etapa é executada separadamente para facilitar o acompanhamento e depuração.

## Importação de Bibliotecas Necessárias
Importamos as bibliotecas essenciais para execução de scripts e validação de arquivos de saída.

In [8]:
# Importação de bibliotecas
import subprocess
import os
import sys
from variaveis import INPUT_PATHS, OUTPUT_PATHS, CARBONO_CONSOLIDADO

## Funções Auxiliares
Definimos funções auxiliares para impressão segura e verificação de arquivos de saída.

In [2]:
# Função para impressão segura
def safe_print(s: str):
    try:
        sys.stdout.write(s)
    except UnicodeEncodeError:
        clean = s.encode(sys.stdout.encoding, errors='replace').decode(
            sys.stdout.encoding)
        sys.stdout.write(clean)

# Função para verificar existência de arquivos
def check_files(files):
    status = True
    for f in files:
        if not os.path.exists(f):
            print(f"[MISSING] {f}")
            status = False
        else:
            print(f"[OK]      {f}")
    return status


# Initialize overall success tracker
all_ok = True

## Etapa 1: Executar `00_extrair_pib_municipal.py`
Executa o script responsável por extrair os dados de PIB municipal e valida os arquivos de saída.

In [3]:
# Etapa 1
script = "00_extrair_pib_municipal.py"
outputs = [OUTPUT_PATHS.pib_ibge_csv]

display(f"\n=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    if not check_files(outputs):
        all_ok = False

'\n=== Running 00_extrair_pib_municipal.py ==='

''

'Checking outputs for 00_extrair_pib_municipal.py:'

[OK]      data/partial/pib_municipal_serra_penitente_ibge.csv


## Etapa 2: Executar `01_extrair_gee_municipal_excel.py`
Executa o script responsável por extrair os dados de GEE municipal e valida os arquivos de saída.

In [4]:
# Etapa 2
script = "01_extrair_cobertura_municipal.py"
outputs = [OUTPUT_PATHS.mapbiomas_long_csv]

display(f"=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    if not check_files(outputs):
        all_ok = False

'=== Running 01_extrair_cobertura_municipal.py ==='

'âœ… CSV long de cobertura MapBiomas gerado em: data/partial/mapbiomas_cobertura_municipal_long.csv\n'

'Checking outputs for 01_extrair_cobertura_municipal.py:'

[OK]      data/partial/mapbiomas_cobertura_municipal_long.csv


## Etapa 3: Executar `02_extrair_alertas_desmatamento.py`
Executa o script responsável por extrair os alertas de desmatamento e valida os arquivos de saída.

In [5]:
# Etapa 3
script = "02_extrair_alertas_desmatamento.py"
outputs = [OUTPUT_PATHS.alertas_csv]

display(f"=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    if not check_files(outputs):
        all_ok = False

'=== Running 02_extrair_alertas_desmatamento.py ==='

'âœ… 1817 alertas salvos em: data/partial/alertas_serra_penitente.csv\n'

'Checking outputs for 02_extrair_alertas_desmatamento.py:'

[OK]      data/partial/alertas_serra_penitente.csv


## Etapa 4: Executar `03_extrair_uso_terra_timeseries.py`
Executa o script responsável por extrair as séries temporais de uso da terra e valida os arquivos de saída.

In [6]:
# Etapa 4
script = "03_extrair_uso_terra_timeseries.py"
outputs = [INPUT_PATHS.uso_timeseries]

display(f"=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    if not check_files(outputs):
        all_ok = False

'=== Running 03_extrair_uso_terra_timeseries.py ==='

'Amostra (longo):\n   codigo_ibge   municipio  uso  year        area_ha     area_km2\n0      1200013  AcrelÃ¢ndia    0  1985      17.611782     0.176118\n1      1200013  AcrelÃ¢ndia    3  1985  170522.562954  1705.225630\n2      1200013  AcrelÃ¢ndia    6  1985    1801.208313    18.012083\n3      1200013  AcrelÃ¢ndia   11  1985      16.031937     0.160319\n4      1200013  AcrelÃ¢ndia   12  1985     244.979835     2.449798\n\nAmostra pÃ³s-filtro:\n   codigo_ibge      municipio  uso  year        area_ha     area_km2\n0      2100501  Alto ParnaÃ\xadba    3  1985   81189.241510   811.892415\n1      2100501  Alto ParnaÃ\xadba    4  1985  846250.420751  8462.504208\n2      2100501  Alto ParnaÃ\xadba    9  1985       0.000000     0.000000\n3      2100501  Alto ParnaÃ\xadba   11  1985    9934.681447    99.346814\n4      2100501  Alto ParnaÃ\xadba   12  1985  167314.721706  1673.147217\n\nAmostra agregada (municÃ\xadpio Ã— uso Ã— ano):\n   codigo_ibge      municipio  year  uso        area_ha    

'Checking outputs for 03_extrair_uso_terra_timeseries.py:'

[OK]      data/partial/uso_terra_serra_penitente_timeseries.csv


## Etapa 5: Executar `04_consolidar_modelar_carbono.py`
Executa o script responsável por consolidar e modelar os dados de carbono e valida os arquivos de saída.

In [10]:
# Etapa 5
script = "04_consolidar_dados_carbono.py"
outputs = [
    CARBONO_CONSOLIDADO,
    OUTPUT_PATHS.model_results_csv
]

display(f"=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    if not check_files(outputs):
        all_ok = False

'=== Running 04_consolidar_dados_carbono.py ==='

'âœ… Dataset final gerado: data/generated/carbono_serra_penitente.csv\nâœ… MÃ©tricas salvas em results/carbon_price_model_all_results.csv\n'

'Checking outputs for 04_consolidar_dados_carbono.py:'

[OK]      data/generated/carbono_serra_penitente.csv
[OK]      results/carbon_price_model_all_results.csv


## Etapa 6: Executar `05_gerar_figuras_carbono.py`
Executa o script responsável por gerar as figuras de carbono e valida os arquivos de saída dinamicamente.

In [11]:
# Etapa 6
script = "05_gerar_figuras_carbono.py"

display(f"=== Running {script} ===")
result = subprocess.run([sys.executable, script], capture_output=True, text=True)
display(result.stdout)
if result.returncode != 0:
    safe_print(f"[ERROR] {script} failed (exit code {result.returncode})\n")
    all_ok = False
else:
    display(f"Checking outputs for {script}:")
    fig_dir = os.path.dirname(OUTPUT_PATHS.evolucao_pib_png)
    expected = []
    for i in range(1, 6):
        expected.append(os.path.join(fig_dir, f"Figura{i:02d}_*.png"))
    for i in range(1, 10):
        expected.append(os.path.join(fig_dir, f"Figura07_{i}_*.png"))
    expected.append(os.path.join(fig_dir, "Figura08_Importancia_Variaveis.png"))
    expected.append(os.path.join(fig_dir, "Figura09_Evolucao_Preco_Carbono.png"))
    import glob
    for pattern in expected:
        matches = glob.glob(pattern)
        if matches:
            display(f"[OK]      Pattern {pattern} -> {len(matches)} file(s)")
        else:
            display(f"[MISSING] Pattern {pattern}")
        all_ok = all_ok and len(matches) > 0

'=== Running 05_gerar_figuras_carbono.py ==='

'[INFO] Carregando dados consolidados de: data/generated/carbono_serra_penitente.csv\n[INFO] DataFrame carregado: 136 linhas, 5 colunas\n[INFO] Carregando sÃ©rie de preÃ§os de carbono de: data/raw/carbon-prices-latest.xlsx\n[INFO] SÃ©rie de preÃ§os carregada: 196 linhas, 45 colunas\n[INFO] PreÃ§os filtrados EU ETS: 7 anos\n[INFO] ApÃ³s merge de preÃ§os: 136 linhas, 6 colunas\n[INFO] Agregando dados por municÃ\xadpio e ano...\n[INFO] DataFrame agregado: 136 registros Ãºnicos\n[INFO] Gerando Figura 01\n[OK] Figura 01 salva em results/figures\\Figura01_Evolucao_PIB.png\n[INFO] Gerando Figura 02\n[OK] Figura 02 salva em results/figures\\Figura02_Evolucao_GEE.png\n[INFO] Gerando Figura 03\n[OK] Figura 03 salva em results/figures\\Figura03_Evolucao_Desmatamento.png\n[INFO] Carregando mÃ©tricas de modelos\n[INFO] MÃ©tricas carregadas: 9 modelos\n[INFO] Gerando Figura 04\n[OK] Figura 04 salva em results/figures\\Figura04_EQM_Modelos.png\n[INFO] Gerando Figura 05\n[OK] Figura 05 salva em result

'Checking outputs for 05_gerar_figuras_carbono.py:'

'[OK]      Pattern results/figures\\Figura01_*.png -> 1 file(s)'

'[OK]      Pattern results/figures\\Figura02_*.png -> 1 file(s)'

...

'[OK]      Pattern results/figures\\Figura07_7_*.png -> 1 file(s)'

'[OK]      Pattern results/figures\\Figura07_8_*.png -> 1 file(s)'

'[OK]      Pattern results/figures\\Figura07_9_*.png -> 1 file(s)'

'[OK]      Pattern results/figures\\Figura08_Importancia_Variaveis.png -> 1 file(s)'

'[OK]      Pattern results/figures\\Figura09_Evolucao_Preco_Carbono.png -> 1 file(s)'

## Conclusão
Após a execução de todas as etapas, verificamos se todos os arquivos de saída esperados foram gerados corretamente.

In [12]:
# Conclusão
# all_ok = result.returncode == 0  # Check if the last step ran successfully
print("Pipeline validation completed.")
if not all_ok:
    print("Some steps failed or outputs are missing.")
    sys.exit(1)
else:
    print("All steps ran successfully and outputs are present.")

Pipeline validation completed.
All steps ran successfully and outputs are present.
